# Investments I: Fundamentals of Performance Evaluation
by Scott Weisbenner University of Illinois at Urbana-Champaign
## Module 1-8 Assignment 2

In [108]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import numpy.testing as npt
import scipy.optimize as opt

def fV_p(w, rho, s):
    """
    Portfolio variance
    :param w: Allocation weights
    :param rho: Asset correlations
    :param s: Variance of asset return
    """
    cov=s@s.T * rho
#     print(f'cov\n{cov}\nw\n{w}')
    V=np.sum(np.dot(cov, w) * w, axis=0)
#     print(f'V\n{np.reshape(V, (1,-1))}')
    return np.reshape(V, (1,-1))

def fr_p(w, r):
    """
    Portfolio return
    :param w: Allocation weights
    :param r: Asset returns
    """
    return np.dot(r.T, w)

def fsharpe(w, r, rho, s):
    """Sharpe ratio"""
    s2=fV_p(w, rho, s)[0,0]
    E=fr_p(w, r)[0,0]
    S=(E-rf)/np.sqrt(s2)
    return S

def print_portfolio(w):
    print(f'Portfolio weights ' + ', '.join(f'{t}:{100*x:.1f}%' for t,x in zip(labels, w[:,0])))
    s2=fV_p(w, rho, s)[0,0]
    print(f'Portfolio variance {s2:.3f}, std. dev. {np.sqrt(s2):.3f}')
    print(f'Return {100*fr_p(w, r)[0,0]:.2f}%')
    S=fsharpe(w, r, rho, s)
    print(f'Sharpe ratio {S:.3f}')

### Model Setup

In [109]:
labels=['large', 'small', 'value', 'growth', 'gold']
# MONTHLY Large, small, value, growth, gold returns
r=np.asfarray([[0.0101, 0.0135, 0.0151, 0.0095, 0.0054]]).T
# MONTHLY Large, small, value, growth, gold std. dev. in returns
s=np.asfarray([[0.043, 0.061, 0.0594, 0.0513, 0.0566]]).T
# Correlations
rho=[
    [1, 0.65, 0.75, 0.93, -0.01],
    [0.65, 1, 0.75, 0.69, 0.07],
    [0.75, 0.75, 1, 0.65, 0.01],
    [0.93, 0.69, 0.65, 1, 0],
    [-0.01, 0.07, 0.01, 0, 1]
]
# Risk-free MONTHLY return rate
rf=0.004

### Q1-1: Portfolio with Same Variance as Large Stocks having Maximum Return
#### Negative weights *not* allowed
### Q1-2: Difference in Return between This Portfolio and Large Stocks

In [110]:
# Note, reuses s2 from previous block
def f(x):
    x=np.reshape(x, (-1,1))
    return -fr_p(x, r)[0,0]
x0=[0.2,0.2,0.2,0.2,0.2]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
    {'type':'eq', 'fun':lambda x: fV_p(x, rho, s)[0,0]-s[0,0]**2}  # Fixed variance
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x

result=opt.minimize(f, x0, method='SLSQP', constraints=c)
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)
npt.assert_allclose(s[0,0]**2, fV_p(w, rho, s)[0,0], rtol=1e-4)
assert fr_p(w, r)[0,0] >= r[0,0]

print_portfolio(w)
diff=fr_p(w, r)[0,0]-r[0,0]
print(f'Difference in return between this portfolio and large stocks {100*diff:.1f}%')

Portfolio weights large:21.8%, small:10.6%, value:48.4%, growth:0.0%, gold:19.2%
Portfolio variance 0.002, std. dev. 0.043
Return 1.20%
Sharpe ratio 0.185
Difference in return between this portfolio and large stocks 0.2%


### Q2: Portfolio with Same Variance as Large Stocks having Maximum Return
#### Negative weights allowed

In [111]:
# Note, reuses s2 from previous block
def f(x):
    x=np.reshape(x, (-1,1))
    return -fr_p(x, r)[0,0]
x0=[0.2, 0.2, 0.2, 0.2, 0.2]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
    {'type':'eq', 'fun':lambda x: fV_p(x, rho, s)[0,0]-s[0,0]**2}  # Fixed variance
)

bounds=[(-1, 2),]*len(r)
# Note that default sampling method does not work
result=opt.shgo(f, bounds, sampling_method='sobol', constraints=c, minimizer_kwargs={'method':'SLSQP'})
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)
npt.assert_allclose(s[0,0]**2, fV_p(w, rho, s)[0,0], rtol=1e-4)
assert fr_p(w, r)[0,0] >= r[0,0]

print_portfolio(w)

Portfolio weights large:111.8%, small:30.5%, value:30.4%, growth:-86.6%, gold:13.9%
Portfolio variance 0.002, std. dev. 0.043
Return 1.25%
Sharpe ratio 0.198


### Q3: Portfolio with Maximum Sharpe Ratio
#### Negative weights *not* allowed

In [112]:
# Setup
def f(x):
    x=np.reshape(x, (-1,1))
    return -fsharpe(x, r, rho, s)  # Maximize
x0=[0.2,0.2,0.2,0.2,0.2]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x

# Optimize
bounds=[(-1, 2),]*len(r)
# Note that default sampling method does not work
result=opt.shgo(f, bounds, sampling_method='sobol', constraints=c, minimizer_kwargs={'method':'SLSQP'})
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)

print_portfolio(w)

Portfolio weights large:0.0%, small:14.9%, value:75.0%, growth:-0.0%, gold:10.1%
Portfolio variance 0.003, std. dev. 0.052
Return 1.39%
Sharpe ratio 0.189


## International Stocks

### Model Setup

In [113]:
labels=['US', 'Japan', 'Asia (ex. Japan)', 'Europe']
# Monthly returns
r=np.asfarray([[0.0089, 0.0021, 0.0096, 0.0074]]).T
# Monthly std. dev. in returns
s=np.asfarray([[0.0434, 0.0597, 0.0601, 0.0504]]).T
# Correlations
rho=[
    [1, 0.41, 0.71, 0.79],
    [0.41, 1, 0.47, 0.50],
    [0.71, 0.47, 1, 0.75],
    [0.79, 0.50, 0.75, 1]
]
# Risk-free monthly return rate
rf=0.0024

### Q4: Portfolio with Same Variance as US Stocks having Maximum Return
#### Negative weights *not* allowed

In [114]:
# Note, reuses s2 from previous block
def f(x):
    x=np.reshape(x, (-1,1))
    return -fr_p(x, r)[0,0]
x0=[0.25, 0.25, 0.25, 0.25]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
    {'type':'eq', 'fun':lambda x: fV_p(x, rho, s)[0,0]-s[0,0]**2}  # Fixed variance
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x

result=opt.minimize(f, x0, method='SLSQP', constraints=c)
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)
npt.assert_allclose(s[0,0]**2, fV_p(w, rho, s)[0,0], rtol=1e-4)
assert fr_p(w, r)[0,0] >= r[0,0]

print_portfolio(w)

Portfolio weights US:93.6%, Japan:0.2%, Asia (ex. Japan):6.2%, Europe:0.0%
Portfolio variance 0.002, std. dev. 0.043
Return 0.89%
Sharpe ratio 0.150


### Q5: Portfolio with Maximum Sharpe Ratio
#### Negative weights *not* allowed

In [115]:
# Setup
def f(x):
    x=np.reshape(x, (-1,1))
    return -fsharpe(x, r, rho, s)  # Maximize
x0=[0.25, 0.25, 0.25, 0.25]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x

# Optimize
bounds=[(-1, 2),]*len(r)
# Note that default sampling method does not work
result=opt.shgo(f, bounds, sampling_method='sobol', constraints=c, minimizer_kwargs={'method':'SLSQP'})
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)

print_portfolio(w)

Portfolio weights US:86.9%, Japan:-0.0%, Asia (ex. Japan):13.1%, Europe:0.0%
Portfolio variance 0.002, std. dev. 0.044
Return 0.90%
Sharpe ratio 0.151


### Q6: Portfolio with Maximum Sharpe Ratio
#### Assume that the expected return for Japanese stocks is 0.96% per month
#### Negative weights *not* allowed

In [116]:
r=np.asfarray([[0.0089, 0.0096, 0.0096, 0.0074]]).T
# Setup
def f(x):
    x=np.reshape(x, (-1,1))
    return -fsharpe(x, r, rho, s)  # Maximize
x0=[0.25, 0.25, 0.25, 0.25]
c=(
    {'type':'eq', 'fun':lambda x: np.sum(x)-1},  # Weights sum to 1
)
for i,_ in enumerate(x0):
    # Note that the extra arg with default value is needed
    # so that the lambda uses the correct scope for i
    c = c + ({'type':'ineq', 'fun':lambda x, i=i: x[i]}, )  # 0 <= x

# Optimize
bounds=[(-1, 2),]*len(r)
# Note that default sampling method does not work
result=opt.shgo(f, bounds, sampling_method='sobol', constraints=c, minimizer_kwargs={'method':'SLSQP'})
#print(result)
assert result.success

w=np.reshape(result.x, (-1,1))

# Sanity checks
npt.assert_allclose(1., np.sum(w), atol=1e-4)

print_portfolio(w)

Portfolio weights US:69.4%, Japan:29.9%, Asia (ex. Japan):0.7%, Europe:0.0%
Portfolio variance 0.002, std. dev. 0.041
Return 0.91%
Sharpe ratio 0.163
